# Fondamenti di elaborazione immagini

## Effettuiamo l'import delle librerie utilizzate nell'esercitazione.

In [ ]:
import cv2
import numpy as np
import matplotlib as mapli
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
%matplotlib inline

Di seguito i riferimenti alle pagine di documentazione, sempre utiliti:

* Rif: [numpy](https://numpy.org/doc/stable/)
* Rif: [opencv](https://docs.opencv.org/)
* Rif: [matplotlib](https://matplotlib.org/stable/index.html)

Aggiungiamo alcune funzioni di utilita' per semplificare la scrittura del codice.

In [ ]:
def rgb(image : np.array) -> np.array:
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def gray(image : np.array) -> np.array:
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def grid(images : list[np.array], rows : int, cols : int, size : int, colors : list[str] = None) -> None:
    fig = plt.figure(figsize=(size,size))
    grid = ImageGrid(fig, 111, nrows_ncols=(rows, cols), axes_pad=0.1)

    if colors is not None:
        counter = 0
        for ax, im in zip(grid, images):
            ax.imshow(im, cmap=colors[counter])
            counter = (counter + 1) % len(colors)
        plt.show()
    else:
        for ax, im in zip(grid, images):
            ax.imshow(im)
        plt.show()

## _Modificare contrasto e luminosita' in un'immagine e' una fra le operazioni piu' utili quando si vuole dare variabilita' ad un set di dati o si vogliono effettuare correzioni post acquisizione dati._

In questo caso si vanno a modificare i valori di ogni pixel secondo la seguente formula: $valore(pixel_{corrente}) = {\alpha}*valore(pixel_{precedente}) + {\beta}$

Possiamo agire direttamente sui valori della matrice, con _numpy_, o utilizzare _opencv_. Seguiamo questa strada:

In [ ]:
img = cv2.imread('./imgs/kitten.png', cv2.IMREAD_COLOR)

Modifichiamo quindi l'immagine caricata per renderla piu' chiara e piu' scura.

In [ ]:
lighter_img = cv2.convertScaleAbs(img, alpha=2, beta=0)     # schiarisco l'immagine.
darker_img = cv2.convertScaleAbs(img, alpha=0.5, beta=100)  # scurisco l'immagine.

grid([rgb(img), rgb(lighter_img), rgb(darker_img)], 1, 3, 20)

Con il metodo _convertScaleAbs_ di _opencv_ e' possibile:

1. Moltiplicare i valori di una matrice per un fattore correttivo.
2. Aggiungere una costante.
3. Prendere il valore assoluto.
4. Saturare il risultato fra 0 e 255.

Tramite queste operazioni e, scegliendo i fattori correttivi adatti, e' possibile ottenere quanto desiderato a livello di contrasto e luminosita'.

* Rif: [convertScaleAbs](https://docs.opencv.org/4.x/d2/de8/group__core__array.html#ga3460e9c9f37b563ab9dd550c4d8c4e7d)

Proviamo ad applicare le stesse modifiche con _numpy_. Per farlo ci sono molti modi; di seguito uno che sfrutta il metodo _vectorize_ di _numpy_ e, successivamente, uno che sfrutta il metodo _map_ di **python**. 

Per prima cosa definiamo la funzione di modifica. 

In [ ]:
def change_brightness_contrast(v, alpha, beta):
    v = abs(v * alpha + beta)
    return 0 if v < 0 else 255 if v > 255 else v

Con il metodo _vectorize_ trasformiamo la funzione in modo tale da permetterle di accettare un array _numpy_ e applicare, per ognuno dei suoi elementi, la funzione stessa.

* Rif: [vectorize](https://numpy.org/doc/stable/reference/generated/numpy.vectorize.html)

In [ ]:
alpha, beta = 2, 0
change_bc_function = np.vectorize(change_brightness_contrast)   # trasformo la funzione.
np_lighter_img = change_bc_function(img, alpha, beta)           # la applico a tutti gli elementi.
np_lighter_img = np_lighter_img.astype(np.uint8)                # riporto la'immagine al formato 8 bit senza segno.

Utilizzando puro **python** e il metodo _map_, facciamo l'altra modifica.

In [ ]:
alpha, beta = 0.5, 100
np_darker_img = np.array(list(map(lambda x : x * alpha + beta, img)))
np_darker_img = np_darker_img.astype(np.uint8)

In [ ]:
grid([rgb(lighter_img), rgb(np_lighter_img), rgb(lighter_img - np_lighter_img)], 1, 3, 20)  # mostriamo i due metodi e la differenza.
grid([rgb(darker_img), rgb(np_darker_img), rgb(darker_img - np_darker_img)], 1, 3, 20)      # mostriamo i due metodi e la differenza.

## _Spesso nei dataset ci si interfaccia con oggetti di interesse che, al modificare di rotazione e scala, mantengono il loro contenuto informativo. Per questo motivo aggiungere variabilita' a questi fattori contribuisce positivamente all'addestramento di modelli di rete._

Per farlo, possiamo sfruttare nuovamente _opencv_.

In [ ]:
img = cv2.imread('./imgs/ball.png', cv2.IMREAD_COLOR)
rows, cols = img.shape[0], img.shape[1]

In [ ]:
image_center = (cols / 2, rows / 2) # punto di rotazione al centro immagine.
custom_rotation_angle = 32.0        # angolo di rotazione personalizzato.
scale = 1.0                         # fattore di scala applicabile all'immagine.

In [ ]:
transformation_matrix = cv2.getRotationMatrix2D(image_center,               # costruisco una matrice di rotazione da applicare all'immagine.
                                                custom_rotation_angle,
                                                scale)

rotated_img_1 = cv2.warpAffine(img, transformation_matrix, (cols, rows))    # applico la rotazione all'immagine

E' possibile ruotare di un angolo custom tramile la funzione _getRotationMatrix2D_, passando un angolo in gradi e il fulcro della rotazione. Il fattore di scala permette di applicare anche una trasformazione alla dimensione ma, per il momento, possiamo trascurarla. Per applicare effettivamente la trasformazione, si utilizza la funzione _warpAffine_ che riassegna nuove coordinate ad ogni pixel applicando la matrice precedentemente calcolata.

* Rif: [getRotationMatrix2D](https://docs.opencv.org/4.x/da/d54/group__imgproc__transform.html#gafbbc470ce83812914a70abfb604f4326)
* Rif: [warpAffine](https://docs.opencv.org/4.x/da/d54/group__imgproc__transform.html#ga0203d9ee5fcd28d40dbc4a1ea4451983)

In [ ]:
rotated_img_2 = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)    # applico una rotazione standard.

E' possibile anche applicare rotazioni standard con la funzione _rotate_ e indicando, con un apposito flag, se ruotare di 90, -90 o 180 gradi.

* Rif: [rotate](https://docs.opencv.org/4.7.0/d2/de8/group__core__array.html#ga4ad01c0978b0ce64baa246811deeac24)
* Rif: [RotateFlags](https://docs.opencv.org/4.7.0/d2/de8/group__core__array.html#ga6f45d55c0b1cc9d97f5353a7c8a7aac2)

In [ ]:
grid([rgb(img), rgb(rotated_img_1), rgb(rotated_img_2)], 1, 3, 20)

## _La scala dell'immagine e' allo stesso modo modificabile._

Possiamo sfruttare il metodo _resize_ di _opencv_ indicando una dimensione di destinazione specifica o un fattore di scala per larghezza e altezza.

* Rif: [resize](https://docs.opencv.org/4.7.0/da/d54/group__imgproc__transform.html#ga47a974309e9102f5f08231edc7e7529d)

In [ ]:
img = cv2.imread('./imgs/kitten.png', cv2.IMREAD_COLOR)

In [ ]:
rescaled_img_1 = cv2.resize(img, (250,260))                 # ridimensiono con dimensioni personalizzate
rescaled_img_2 = cv2.resize(img, (0, 0), fx=1.5, fy=1.5)    # ridimensiono con dei fattori di scala.

grid([rgb(img), rgb(rescaled_img_1), rgb(rescaled_img_2)], 1, 3, 20)

## _Quanto detto per la rotazione vale anche per il flip immagine che, nuovamente, puo' essere effettuato tramite opencv._

Il metodo utilizzato e' _flip_ e gli argomenti richiesti sono l'immagine e il tipo di flip:

1. Asse orizzontale: 0
2. Asse verticale: 1
3. Entrambi: -1

* Rif: [flip](https://docs.opencv.org/4.x/d2/de8/group__core__array.html#gaca7be533e3dac7feb70fc60635adf441)

In [ ]:
img = cv2.imread('./imgs/ball.png')
images_to_show = [rgb(img), rgb(cv2.flip(img, 0)), rgb(cv2.flip(img, 1)), rgb(cv2.flip(img, -1))]

grid(images_to_show, 1, 4, 20)

## _Un utile modifica alle immagini che ne permette una modifica impercettibile pur mantenendo il contenuto informativo e' la traslazione._

Quando in un dataset immagini ci si trova di fronte ad una classe poco rappresentata, e' possibile applicare spostamenti casuali di un numero contenuto di pixel. Effettuando questa data augmentation, l'immagine rimane sensata ma a tutti gli effetti, per la rete, e' una matrice di numeri totalmente diversa. In _opencv_ e' possibile sfruttare nuovamente la matrice di trasformazione e il metodo _warpAffine_ indicando il numero di pixel di spostamento che si desidera ottenere in x e in y.

In [ ]:
img = cv2.imread('./imgs/ball.png', cv2.IMREAD_COLOR)                      # carico l'immagine.
rows, cols = img.shape[0], img.shape[1]                                     # memorizzo le sue dimensioni.

transformation_matrix = np.float32([[1,0,50],[0,1,25]])                     # eseguo la traslazione in x di 50 pixel.
                                                                            # eseguo la traslazione in y di 25 pixel.
                                                                            
translated_image = cv2.warpAffine(img, transformation_matrix, (cols, rows)) # applico la trasformazione all'immagine.

grid([rgb(img), rgb(translated_image)], 1, 2, 10)

## _Come visto con gli istogrammi, le immagini possono possedere gruppi di pixel facilmente separabili tramite l'utilizzo di una o piu' soglie secche che ne vanno a distinguere i valori_

Questa operazione si chiama **sogliatura** ed e' una delle operazioni fondamentali, sopratutto quando si vanno ad utilizzare immagini im scala di grigi e maschere. Utilizziamo nuovamente _opencv_ e la funzione _threshold_ per vedere come, al variare delle opzioni, varia il risultato finale. Per farlo, sfruttiamo un'immagine personalizzata dove sono presenti due oggetti distinti con valori di grigio altrettanto distinti.

* Rif: [threshold](https://docs.opencv.org/4.x/d7/d1b/group__imgproc__misc.html#gae8a4a146d1ca78c626a53577199e9c57)

In [ ]:
img = cv2.imread('./imgs/thresholds/levels.png')
binary_thresholds = [img,
    cv2.threshold(img, 0, 255, cv2.THRESH_BINARY)[1],       # i valori > di 0 diventano 255, gli altri 0
    cv2.threshold(img, 0, 50, cv2.THRESH_BINARY)[1],        # i valori > di 0 diventano 50, gli altri 0
    cv2.threshold(img, 90, 255, cv2.THRESH_BINARY)[1],      # i valori > di 90 diventano 255, gli altri 0
]

grid(binary_thresholds, 1, 4, 20)

La soglia binaria esclude tutti pixel con valore inferiore alla soglia e agli altri assegna invece il valore indicato. Nel precedente esempio siamo riusciti a:

1. Considerare tutti i pixel con valore sopra a 0 e assegnargli 255.
2. Considerare tutti i pixel con valore sopra a 0 e assegnargli 50.
3. Considerare tutti i pixel con valore sopra a 90 e assegnargli 255.

In [ ]:
img = cv2.imread('./imgs/thresholds/levels.png')
inverted_binary_thresholds = [img,
    cv2.threshold(img, 90, 255, cv2.THRESH_BINARY_INV)[1],  # i valori > di 90 diventano 0, gli altri 255
]

grid(inverted_binary_thresholds, 1, 2, 10)

La soglia binaria invertita assegna 0 a tutti i pixel che superano la soglia e il valore indicato agli altri. Siamo riusciti infatti a:

1. Azzerare tutti i pixel con valore sopra 90 e dare valore 255 agli altri.

In [ ]:
img = cv2.imread('./imgs/thresholds/levels.png')
truncation_thresholds = [img,
    cv2.threshold(img, 255, 0, cv2.THRESH_TRUNC)[1],         # i valori > di 50 valgono 50, gli altri invariati
]

grid(truncation_thresholds, 1, 2, 10)

In [ ]:
img = cv2.imread('./imgs/thresholds/levels.png')
to_zero_thresholds = [img,
    cv2.threshold(img, 90, 255, cv2.THRESH_TOZERO)[1],      # i valori < 90 valgono 0, gli altri invariati
]

grid(to_zero_thresholds, 1, 2, 10)

Con la soglia a zero, i valori dei pixel si mantengono se sopra soglia altrimenti gli viene assegnato 0. Siamo riusciti quindi a:

1. Azzerare tutti i valori sotto a 90 e mantenere i valori degli altri pixel.

In [ ]:
img = cv2.imread('./imgs/thresholds/levels.png')
to_zero_thresholds = [img,
    cv2.threshold(img, 90, 255, cv2.THRESH_TOZERO_INV)[1],  # i valori > di 90 valgono 0, gli altri invariati
]

grid(to_zero_thresholds, 1, 2, 10)

Con la soglia a zero invertita, i valori dei pixel si azzerano se sopra soglia altrimenti il loro valore rimane invariato. Siamo riusciti quindi a:

1. Azzerare tutti i valori sopra a 90 e mantenere i valori degli altri pixel.

Oltre a questi, esistono ulteriori metodi di sogliatura: a doppia soglia, adattivi...Di seguito alcuni riferimenti:

* Rif: [inRange](https://docs.opencv.org/4.x/d2/de8/group__core__array.html#ga48af0ab51e36436c5d04340e036ce981)
* Rif: [adaptiveThreshold](https://docs.opencv.org/4.x/d7/d1b/group__imgproc__misc.html#ggaa9e58d2860d4afa658ef70a9b1115576a19120b1a11d8067576cc24f4d2f03754)